# attempting to parse usbank q1 2024 earnings supplement


This notebook is focused on exploring tools to parse pdfs and interact with them with an LLM.   
I choose not to focus on word documents because they can be converted to pdfs. 


In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
path_earnings_supplement = "../example_data/Q1-2024-Earnings-Supplement.pdf"
path_earnings_release = "../example_data/Q1-2024-Earnings-Release.pdf "
path_to_doc = path_earnings_supplement

# exploring pdf parsing libs

In [57]:
import pdfplumber
# exploring pdfplumber library

In [58]:

with pdfplumber.open(path_to_doc) as pdf:
    snd_page = pdf.pages[1]
    tables = snd_page.find_tables(None)


1


In [64]:
with pdfplumber.open(path_to_doc) as pdf:
    im = pdf.pages[1].to_image(resolution=150)

im.show()


In [65]:
tables = snd_page.find_tables()
snd_page.extract_table()


[['QUARTERLY CONSOLIDATED STATEMENT OF INCOME', None, None],
 ['(Dollars and Shares in Millions, Except Per Share Data)\n(Unaudited)',
  'March 31, December 31, September 30, June 30,\n2024 2023 2023 2023',
  'March 31,\n2023']]

It seems that most pdf parsers have issues when data and text are presented side be side. It cannot easily distinguish where a table or text starts or ends
Some next steps would be to look and see if their are any other libraries that do better at this, or if we can bulid a system to automatically break up a pdf into smaller, independent chunks

# exploring langchain libs

although the pdf parsers are having trouble answering questiosn, we will still see if they are good enough at extracting information from pdfs  
following [this youtube video](https://www.youtube.com/watch?v=au2WVVGUvc8)/ this [colab notebooks](https://colab.research.google.com/drive/1OZpmLgd5D_qmjTnL5AsD1_ZJDdb7LQZI?usp=sharing)

In [83]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
#from langchain_openai import OpenAIEmbeddings
#from langchain_openai import OpenAI

In [84]:
#PyPDFLoader seems to have trouble with tables
loader = PyPDFLoader(path_to_doc)
pages = loader.load_and_split()
print(pages[0])

# SKIP TO STEP 2 IF YOU'RE USING THIS METHOD
chunks = pages

page_content='Supplemental Consolidated \nSchedules\nFirst  Quarter  2024' metadata={'source': '../example_data/Q1-2024-Earnings-Supplement.pdf', 'page': 0}


In [85]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)

In [86]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
# i occasionally get errors regarding context size
# using different models can also lead to errors with 

query = "What was the interest income from loans on March 31 2024?"
docs = db.similarity_search(query)

chain.run(input_documents=docs, question=query)

' The interest income from loans on March 31, 2024 was $5,712 million.'

In [88]:
query = "What was the amount of total assets in December 31, 2023? "
docs = db.similarity_search(query)

chain.run(input_documents=docs, question=query)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4750 tokens (4494 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Both of these answers seem to be correct from

# conclusion
pdf parsing and token limits seem to be the main bottlenecks for this process
* llms were able to accurately retrieve information from the documents 
    * albeit under generous circumstances, where they only had to query one document and was told explicitly what to look for

next steps
* look more into pdf parsing tools to handle when documents contain both text and data
    * one idea is to pass pdfs through a tool to split the documents into their individual components (CV to create boxes of independent data -> make each box its own pdf)? 
* investigate into using llm to turn pdf data into an actual queriable form:sql db, excel, json
    * if I can convert a pdf to any of these formats, it will be easy to convert between them
